In [ ]:
def weightage(word,text,number_of_documents=1):
    
    number_of_times_word_appeared = len(word_list)

    tf = number_of_times_word_appeared/float(len(text))

    idf = np.log((number_of_documents)/float(number_of_times_word_appeared))
    
    tf_idf = tf*idf
    
    return number_of_times_word_appeared, tf, idf, tf_idf 

In [ ]:
#brauch ich das noch?
def word_count(text):
    counts = dict()
    words = text.split(" ")

    #print(words)
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return words, counts

In [ ]:
cv=CountVectorizer(max_df=3, min_df=0, stop_words=stop_words, max_features=10000, ngram_range=(1,3))
fittedStopwords = ["dummy" for dummy in range(len(corpus))]
for i in range(len(corpus)):
    corpus[i] = [corpus[i]]
    fittedStopwords[i] = cv.fit_transform(corpus[i])
print(fittedStopwords)

In [ ]:

#Iteration über Texte
for i in range(len(corpus)):
    df_keys = {'keywords': [], #Erstellung Dictionary für nachher DataFrame
              'number_of_times_word_appeared': [],
              'idf': [],
              'tf': [],
              'tf_idf': []}
    for keyword in keywords[i]: #Iteration über keywords pro Text
        word_list = re.findall(keyword,corpus[i]) #Finde die keywords wieder im Text
        if len(word_list) >= 1: #Falls keyword gefunden
            curArray = []
            curArray = weightage(keyword, corpus[i])    
            
            #if keyword not in df_keys.get('keywords') : #funktioniert so nicht
            #Füllung des Dictionary anhand der Funktion weightage
            df_keys['keywords'].append([keyword])
            df_keys['number_of_times_word_appeared'].append(curArray[0])
            df_keys['tf'].append(curArray[1])
            df_keys['idf'].append(curArray[2])
            df_keys['tf_idf'].append(curArray[3])
#Umwandlung in Dataframe
    df = pd.DataFrame(df_keys)
    
    df=df.loc[df.astype(str).drop_duplicates(subset='keywords').index]
    df = df.sort_values('tf_idf',ascending=True)
    df.head(25).to_csv(str(i)+'_Keywords.csv') #Erstellung csv pro Text

In [ ]:
from transformers import pipeline

# Output labels
candidate_labels = ["Topic", "No Topic"]

# device=0 for GPU usage
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=0)

# Sample code to see labels for first five rows in df
for i in range(len(corpus)):
  input_text = corpus[i]
  
  # multi_label=True will return confidence score for both labels independently 
  model_dict = classifier(input_text, candidate_labels, multi_label=True)

  # Zip results to dict
  result_dict = dict(zip(model_dict.get('labels'), model_dict.get('scores')))
  
  # Print confidence scores
  print("Input Text for the model : ", input_text)
  print("Confidence Score for Topic Class : ", result_dict.get('Topic'))
  print("Confidence Score for No-Topic Class : ", result_dict.get('No Topic'), end='\n\n')